# Contents
- [Imports](#imports)
- [Visual Test](#vid)
- [Quantitative Test](#quantitative)
    - [Classification on Test Keypoints](#keypointclass)

---
# Imports<a id=imports></a>

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from inspect import signature
from itertools import cycle
from IPython.display import HTML
from IPython.display import Image
import pickle
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import recall_score, make_scorer, roc_curve, classification_report, precision_recall_curve, roc_auc_score

import xgboost as xgb
from xgboost import XGBClassifier

import argparse
import logging
import time
import os
import re
import cv2
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

Using TensorFlow backend.
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chang\.conda\envs\opencv\lib\site-packages\t

In [6]:
#run this to prevent GPU process issues
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

---
# Visual Test<a id=vid></a>
Let us import the necessary code for our video tests.

In [2]:
logger = logging.getLogger('TfPoseEstimator-Video')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

fps_time = 0

In [3]:
parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
parser.add_argument('--video', type=str, default='')
parser.add_argument('--resolution', type=str, default='432x368', help='network input resolution. default=432x368')
parser.add_argument('--model', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin / mobilenet_v2_large / mobilenet_v2_small')
parser.add_argument('--show-process', type=bool, default=False,
                    help='for debug purpose, if enabled, speed for inference is dropped.')
parser.add_argument('--showBG', type=bool, default=True, help='False to show skeleton only.')

_StoreAction(option_strings=['--showBG'], dest='showBG', nargs=None, const=None, default=True, type=<class 'bool'>, choices=None, help='False to show skeleton only.', metavar=None)

Our testing function has to be slightly altered to account for the engineered features that we have to create on the fly.<br/>
The rest of the function stays the same.

In [4]:
def vidtest(args):
    
    fps_time = 0
    
    lis=[]
    for x in range(1,18):
        lis.append('x'+str(x))
        lis.append('y'+str(x))
    header=pd.DataFrame(columns=lis)
    
    xparts=['x1','x2','x3','x4','x5','x6','x7','x8','x11']
    yparts=['y1','y2','y3','y4','y5','y6','y7','y8','y11']
    drop=['x0', 'x10', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x9', 'y0', 'y10', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y9']
    
    logger.debug('initialization %s : %s' % (args.model, get_graph_path(args.model)))
    w, h = model_wh(args.resolution)
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))
    cap = cv2.VideoCapture(args.video)

    if cap.isOpened() is False:
        print("Error opening video stream or file")
    while cap.isOpened():
        try:
            ret_val, image = cap.read()
            humans = e.inference(image, upsample_size=4.0)
        except:
            break
            
        if not args.showBG:
            image = np.zeros(image.shape)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

        cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        move='none'

        try:
            part=re.findall('BodyPart:(.*?)-',str(humans[0]))
            xco=re.findall('\((.*?),',str(humans[0]))
            yco=re.findall(', (.*?)\)',str(humans[0]))
            xpart=['x'+c for c in part]
            ypart=['y'+c for c in part]
            test1=pd.DataFrame([[float(x) for x in xco+yco]],columns=xpart+ypart)
            data=pd.concat([header,test1],sort=True)

            data.drop(columns=drop,inplace=True) #create our new features based on our previous feature engineering
            data[xparts]=data[xparts].subtract(data['x1'],axis=0)
            data[yparts]=data[yparts].subtract(data['y1'],axis=0)
            data.fillna(0,inplace=True)
    
            data['shwidth']=np.hypot((data.x2-data.x5),(data.y2-data.y5))
            data['splength']=np.hypot((data.x1-((data.x8+data.x11)/2)),(data.y1-((data.y8+data.y11)/2)))
            data['lhnsh']=np.hypot((data.x7-data.x5),(data.y7-data.y5))*data.shwidth/data.splength
            data['rhnsh']=np.hypot((data.x4-data.x2),(data.y4-data.y2))*data.shwidth/data.splength
            data['lelsh']=np.hypot((data.x6-data.x5),(data.y6-data.y5))*data.shwidth/data.splength
            data['relsh']=np.hypot((data.x3-data.x2),(data.y3-data.y2))*data.shwidth/data.splength #end of feature creation
            
            predict=mod.predict(data.to_numpy())
            if predict[0]==0:
                move='none'
            elif predict[0]==1:
                move='guard'
            elif predict[0]==2:
                move='jab'
            elif predict[0]==3:
                move='cross'
            elif predict[0]==4:
                move='hook'
            elif predict[0]==5:
                move='uppercut'
            else:
                move='none'

        except:
            move='not detected'

        finally:
            cv2.putText(image,
                        'move: '+move,
                        (10, 30),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 0, 255), 2)
            out.write(image)
            cv2.imshow('tf-pose-estimation result', image)
            fps_time = time.time()

            if cv2.waitKey(1) == 27:
                break
            
    cv2.destroyAllWindows()
    logger.debug('finished+')
    return(data)

Again, we set our output video codec to 'mp4v'.

In [5]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

And we perform our estimation, prediction and output with the loop below.

In [10]:
path='./data/model2/'
args=parser.parse_args(("--video=../results/test.mp4 --model=cmu").split())
for e_name in os.listdir(path):
    model=os.path.splitext(e_name)[0]
    writepath="../results/test2/"+str(model)+".mp4"
    out=cv2.VideoWriter(writepath, fourcc, 30, (1280,720))
    mod=pickle.load(open(r'./data/model2/'+e_name, 'rb'))
    vidtest(args)

[2019-08-01 01:19:57,218] [TfPoseEstimator-Video] [DEBUG] initialization cmu : D:\Insync\Team Drive\amilame@gmail.com\amilame@gmail.com\HAR\tf-pose-estimation\models\graph/cmu/graph_opt.pb
[2019-08-01 01:19:57,218] [TfPoseEstimator-Video] [DEBUG] initialization cmu : D:\Insync\Team Drive\amilame@gmail.com\amilame@gmail.com\HAR\tf-pose-estimation\models\graph/cmu/graph_opt.pb
I0801 01:19:57.218427 16048 <ipython-input-8-5cee673a3860>:15] initialization cmu : D:\Insync\Team Drive\amilame@gmail.com\amilame@gmail.com\HAR\tf-pose-estimation\models\graph/cmu/graph_opt.pb
[2019-08-01 01:19:57,222] [TfPoseEstimator] [INFO] loading graph from D:\Insync\Team Drive\amilame@gmail.com\amilame@gmail.com\HAR\tf-pose-estimation\models\graph/cmu/graph_opt.pb(default size=432x368)
I0801 01:19:57.222390 16048 estimator.py:309] loading graph from D:\Insync\Team Drive\amilame@gmail.com\amilame@gmail.com\HAR\tf-pose-estimation\models\graph/cmu/graph_opt.pb(default size=432x368)
[2019-08-01 01:27:47,421] [Tf

KeyboardInterrupt: 

The results are shown below.

In [2]:
def showvideo(i):
    display(HTML("""<video width="1500" height="1000" controls><source src="{}" type="video/mp4"></video>""".format(i)))

path='./data/model2/'
for e_name in os.listdir(path):
    model=os.path.splitext(e_name)[0]
    print('result of '+str(model)+':')
    showvideo('../results/test2/'+str(model)+'.mp4')

result of ADA:


result of bagging classifier:


result of DecisionTree:


result of ef:


result of GBoost:


result of knn:


result of lr with reg:


result of random forest:


result of SVC:


result of XGB:


Through visual inspection, we get much better results with our newly trained models.<br/>
Taking Gradient Boosting and SVC videos as an example, the predictions are much more stable and less jumpy as compared to our first round of testing.<br/>
The predictors are also much better at predicting the sequence of jab, cross, hook and uppercut movements.

---
# Quantitative Test<a id=quantitative></a>

## Classification on Test Keypoints<a id=keypointclass></a>
We utilize the same keypoint data extract in our first round of testing.<br/>
We display the classification reports and observe the accuracy scores.

In [8]:
none=pd.read_csv('../results/test/none.csv')
guard=pd.read_csv('../results/test/guard.csv')
jab=pd.read_csv('../results/test/jab.csv')
cross=pd.read_csv('../results/test/cross.csv')
hook=pd.read_csv('../results/test/hook.csv')
upcut=pd.read_csv('../results/test/uppercut.csv')

In [9]:
none['move']=0
guard['move']=1
jab['move']=2
cross['move']=3
hook['move']=4
upcut['move']=5

In [10]:
data=pd.concat([none,guard,jab,cross,hook,upcut],sort=True)
data.head()

,move,x0,x1,x10,x11,x12,x13,x14,x15,x16,...,y16,y17,y2,y3,y4,y5,y6,y7,y8,y9
0,0,0.75,0.69,NaN,0.63,0.67,NaN,0.75,NaN,0.71,...,0.12,NaN,0.26,0.54,0.79,0.27,NaN,NaN,0.68,0.90
1,0,0.75,0.68,NaN,0.62,0.63,NaN,0.75,NaN,0.71,...,0.13,NaN,0.26,0.53,0.79,0.27,NaN,NaN,0.67,0.88
2,0,0.74,0.68,NaN,0.63,0.66,NaN,0.74,NaN,0.70,...,0.13,NaN,0.26,0.52,0.79,0.27,NaN,NaN,0.67,0.89
3,0,0.74,0.67,NaN,0.62,0.64,NaN,0.74,NaN,0.70,...,0.14,NaN,0.26,0.52,0.78,0.27,NaN,NaN,0.69,0.91
4,0,0.73,0.67,NaN,0.63,NaN,NaN,0.73,NaN,0.69,...,0.14,NaN,0.27,0.52,0.78,0.27,NaN,NaN,0.69,0.91


In this run of testing, we have to create the features we created in our feature engineering section.

In [11]:
xparts=['x1','x2','x3','x4','x5','x6','x7','x8','x11']
yparts=['y1','y2','y3','y4','y5','y6','y7','y8','y11']
drop=['x0', 'x10', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x9', 'y0', 'y10', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y9']
data.drop(columns=drop,inplace=True)

data[xparts]=data[xparts].subtract(data['x1'],axis=0)
data[yparts]=data[yparts].subtract(data['y1'],axis=0)
data.fillna(0,inplace=True)

data['shwidth']=np.hypot((data.x2-data.x5),(data.y2-data.y5))
data['splength']=np.hypot((data.x1-((data.x8+data.x11)/2)),(data.y1-((data.y8+data.y11)/2)))
data['lhnsh']=np.hypot((data.x7-data.x5),(data.y7-data.y5))*data.shwidth/data.splength
data['rhnsh']=np.hypot((data.x4-data.x2),(data.y4-data.y2))*data.shwidth/data.splength
data['lelsh']=np.hypot((data.x6-data.x5),(data.y6-data.y5))*data.shwidth/data.splength
data['relsh']=np.hypot((data.x3-data.x2),(data.y3-data.y2))*data.shwidth/data.splength

In [12]:
data.head()

,move,x1,x11,x2,x3,x4,x5,x6,x7,x8,...,y5,y6,y7,y8,shwidth,splength,lhnsh,rhnsh,lelsh,relsh
0,0,0.0,-0.06,0.02,0.08,0.17,-0.03,0.0,0.0,0.02,...,0.01,0.0,0.0,0.42,0.050990,0.420476,0.003835,0.066796,0.003835,0.034726
1,0,0.0,-0.06,0.02,0.07,0.17,-0.02,0.0,0.0,0.02,...,0.01,0.0,0.0,0.41,0.041231,0.405494,0.002274,0.056008,0.002274,0.027921
2,0,0.0,-0.05,0.02,0.07,0.16,-0.02,0.0,0.0,0.00,...,0.01,0.0,0.0,0.41,0.041231,0.405771,0.002272,0.055701,0.002272,0.026903
3,0,0.0,-0.05,0.02,0.06,0.15,-0.02,0.0,0.0,0.01,...,0.00,0.0,0.0,0.42,0.041231,0.415482,0.001985,0.053191,0.001985,0.026105
4,0,0.0,-0.04,0.02,0.05,0.14,-0.02,0.0,0.0,0.01,...,0.00,0.0,0.0,0.42,0.040000,0.415271,0.001926,0.050466,0.001926,0.024253


There seem to be no issues with our dataset. Let us proceed with the testing.

In [13]:
X=data[[x for x in data.columns if x != 'move']].to_numpy()
y=data['move']

And our test results are given below:

In [14]:
path='./data/model2/'
for e_name in os.listdir(path):
    model=os.path.splitext(e_name)[0]
    estimator=pickle.load(open(r'./data/model2/'+e_name, 'rb'))
    pred=estimator.predict(X)
    print('Results for ',model,':')
    print(classification_report(y,pred))

Results for  ADA :
              precision    recall  f1-score   support

           0       0.71      0.63      0.67       926
           1       0.97      0.34      0.50      2271
           2       0.05      0.14      0.07       281
           3       0.69      0.30      0.42       195
           4       0.18      0.36      0.24       172
           5       0.13      0.99      0.22       155

    accuracy                           0.42      4000
   macro avg       0.45      0.46      0.35      4000
weighted avg       0.76      0.42      0.48      4000

Results for  bagging classifier :
              precision    recall  f1-score   support

           0       0.67      0.68      0.68       926
           1       0.99      0.35      0.51      2271
           2       0.75      0.34      0.47       281
           3       0.09      0.63      0.16       195
           4       0.33      0.60      0.42       172
           5       0.30      0.90      0.45       155

    accuracy            

We get much better results for **certain** models, with Linear Regression having an accuracy of 76% and SVC having an accuracy of 79% as compared to our previous maximum accuracy of 65% for Gradient Boosting.<br/>

The rest of our models seem to have lower test scores though this seems to be misleading as our visual test shows our models to actually perform better. The prediction accuracy may be lower overall, but predictions seem to be accurate in the important phases of the movements.